In [1]:
import sys, os, pathlib
#setting the path to folder with modules
sys.path.insert(0, str(pathlib.Path(os.getcwd()).parents[1] / 'python'))
from Load_Datasets import *

In [130]:
FileAddress_movies ="../../Datasets/tmdb_5000_movies.csv"
FileAddress_credits="../../Datasets/tmdb_5000_credits.csv"

In [ ]:
#Loading information about movies
Movies,Credentials  = Load_Datasets(FileAddress_movies,FileAddress_credits)

Drops = ['homepage','status','id']
for drop in Drops:
    Movies = Movies.drop(drop, 1)
        
    

Movies.to_csv("../../Datasets/Transformed.csv")
#print(Movies.dtypes)
#read arrays from string using string.split(",")

In [4]:
Movies

budget  \
title                                                 
#Horror                                     1500000   
(500) Days of Summer                        7500000   
10 Cloverfield Lane                        15000000   
10 Days in a Madhouse                       1200000   
10 Things I Hate About You                 16000000   
102 Dalmatians                             85000000   
10th & Wolf                                 8000000   
11:14                                       6000000   
12 Angry Men                                 350000   
12 Rounds                                  20000000   
12 Years a Slave                           20000000   
127 Hours                                  18000000   
13 Going on 30                             37000000   
13 Hours: The Secret Soldiers of Benghazi  50000000   
1408                                       25000000   
15 Minutes                                 60000000   
16 Blocks                                  55000000   
16 to Life                                        0   
17 Again                                   20000000   
1776                                              0   
1911                                       30000000   
1941                                       35000000   
1982                                        1000000   
2 Fast 2 Furious                           76000000   
2 Guns                                     61000000   
20 Dates                                          0   
20 Feet from Stardom                        1000000   
20,000 Leagues Under the Sea                5000000   
200 Cigarettes                              6000000   
2001: A Space Odyssey                      10500000   
...                                             ...   
Young Adult                                12000000   
Young Frankenstein                          2800000   
Young Guns                                 13000000   
Young Sherlock Holmes                      18000000   
Your Highness                              49900000   
Your Sister's Sister                         125000   
Yours, Mine and Ours                              0   
Youth in Revolt                            18000000   
Z Storm                                           0   
ZMD: Zombies of Mass Destruction                  0   
Zack and Miri Make a Porno                 24000000   
Zambezia                                   20000000   
Zathura: A Space Adventure                 65000000   
Zero Dark Thirty                           40000000   
Zero Effect                                 5000000   
Zipper                                            0   
Zodiac                                     65000000   
Zombie Hunter                                     0   
Zombieland                                 23600000   
Zookeeper                                  80000000   
Zoolander                                  28000000   
Zoolander 2                                50000000   
Zoom                                       35000000   
Zulu                                       16000000   
[REC]                                       1500000   
[REC]²                                      5600000   
eXistenZ                                   15000000   
xXx                                        70000000   
xXx: State of the Union                    60000000   
Æon Flux                                   62000000   

                                                                                      genres  \
title                                                                                          
#Horror                                                        Drama,Mystery,Horror,Thriller   
(500) Days of Summer                                                    Comedy,Drama,Romance   
10 Cloverfield Lane                                           Thriller,Science Fiction,Drama   
10 Days in a Madhouse                                                                  Drama   
10 

In [217]:
def cost_function(current_index):
    """Define the cost between the film given in index and the others one."""
    
    costs = np.zeros(Movies.shape[0])
    
    current_film = Movies.iloc[current_index]
    genres_current = get_genres(current_film)
    kw_current = get_keywords(current_film)
    current_overview = text_mining(current_film)
    
    for i in range(Movies.shape[0]):
        cost = 0
        
        b_film = Movies.iloc[i]
        genres_b = get_genres(b_film)
        #First we only select the first genre to determine the similarity because it's more important that the other genre.
        if len(genres_current) > 0  & len(genres_b) > 0:
            if (genres_current[0] == genres_b[0]):
                cost += 2

            #This give us the number of similar genres. We pop the first one because we already compare them.
            cost += np.sum(np.in1d(genres_current,genres_b.pop(0),assume_unique='True')) / 10
            #TODO: Do the same for secondary genre but with less weight.


        kw_b = get_keywords(b_film)
        #This give us the number of similar keywords.
        cost += np.sum(np.in1d(kw_current,kw_b,assume_unique='True'))
        
        overview_b = text_mining(b_film)
        
        #Compute the cosine distances between the two texts.
        cosine = get_cosine(current_overview, overview_b)
        
        cost += cosine
        costs = cost + cosine
    return costs

def get_genres(film):
    genres = str(film['genres'])
    if genres == 'nan':
        return[]
    else:
        genres = genres.split(",")
    return genres

def get_keywords(film):
    kw = str(film['keywords'])
    if kw == 'nan':
        return[]
    else:
        kw = kw.split(",")
    return kw

def text_mining(film):
    stop_words = ['all', 'just', 'being', 'over', 'both', 'through', 'yourselves', 'its', 'before', 'herself', 'had', 'should', 'to', 'only', 'under', 'ours', 'has', 'do', 'them', 'his', 'very', 'they', 'not', 'during', 'now', 'him', 'nor', 'did', 'this', 'she', 'each', 'further', 'where', 'few', 'because', 'doing', 'some', 'are', 'our', 'ourselves', 'out', 'what', 'for', 'while', 'does', 'above', 'between', 't', 'be', 'we', 'who', 'were', 'here', 'hers', 'by', 'on', 'about', 'of', 'against', 's', 'or', 'own', 'into', 'yourself', 'down', 'your', 'from', 'her', 'their', 'there', 'been', 'whom', 'too', 'themselves', 'was', 'until', 'more', 'himself', 'that', 'but', 'don', 'with', 'than', 'those', 'he', 'me', 'myself', 'these', 'up', 'will', 'below', 'can', 'theirs', 'my', 'and', 'then', 'is', 'am', 'it', 'an', 'as', 'itself', 'at', 'have', 'in', 'any', 'if', 'again', 'no', 'when', 'same', 'how', 'other', 'which', 'you', 'after', 'most', 'such', 'why', 'a', 'off', 'i', 'yours', 'so', 'the', 'having', 'once']
    
    overview = str(film['overview'])
    if overview == 'nan':
        return Counter([])
    
    words = [i for i in film['overview'].lower().split() if i not in stop_words]
    for word in words:
        word = stem(word)

    return Counter(words)

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator


In [218]:
costs = np.zeros([Movies.shape[0],Movies.shape[0]])
for i in tqdm(range(0,Movies.shape[0])):
    costs[i] = cost_function(i)
print(costs)

  3%|▎         | 148/4803 [22:17<11:41:01,  9.04s/it]

KeyboardInterrupt: 

In [136]:
#We normalize the matrix
costs = (costs - np.mean(costs)) / np.std(costs)
print(costs)

[[ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]]


/Users/clement/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [138]:
Movies = pd.read_csv("../../Datasets/Transformed.csv")

In [160]:
film = Movies.iloc[71]
kw = str(film['keywords'])
if kw == 'nan':
    print('a')
else:
    kw.split(",")

a
